In [ ]:
from exp.run import run_experiment
from exp.mappings import alg_map
from exp.train import train_model
from exp.features import load_cv_results
import pandas as pd
import json
import numpy as np
import os
import pickle
import inspect
import kaggle_files

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

### Load results from CSV File and re-produce models ranked by CV scores

In [ ]:
save_results= "exp1.csv"
score_df = load_cv_results(save_results)
score_df['filename'] = save_results

save_results= "exp2.csv"
tmp = load_cv_results(save_results)
tmp['filename'] = save_results
score_df = score_df.append(tmp)

save_results= "exp3.csv"
tmp = load_cv_results(save_results)
tmp['filename'] = save_results
score_df = score_df.append(tmp)
score_df["feature_set"] = "standard_scaled"

save_results= "exp4_1.csv"
score_df_ = load_cv_results(save_results)
score_df_['filename'] = save_results
score_df_ = score_df_.rename(index=str, columns={"cv_score_n_folds_10_shuffle_True_rs_41": "score"})
score_df_['n_folds'] = 10
score_df_['shuffle'] = True
score_df_['rs'] = 41
score_df = score_df.append(score_df_)

save_results= "exp5.csv"
score_df_ = load_cv_results(save_results)
score_df_['filename'] = save_results
score_df_ = score_df_.rename(index=str, columns={"cv_score_n_folds_10_shuffle_True_rs_41": "score"})
score_df_['n_folds'] = 10
score_df_['shuffle'] = True
score_df_['rs'] = 41
score_df = score_df.append(score_df_)

save_results= "exp6.csv"
score_df_ = load_cv_results(save_results)
score_df_['filename'] = save_results
score_df_ = score_df_.rename(index=str, columns={"cv_score_n_folds_10_shuffle_True_rs_41": "score"})
score_df_['n_folds'] = 10
score_df_['shuffle'] = True
score_df_['rs'] = 41
score_df = score_df.append(score_df_)

save_results= "exp7.csv"
score_df_ = load_cv_results(save_results)
score_df_['filename'] = save_results
score_df_ = score_df_.rename(index=str, columns={"cv_score_n_folds_10_shuffle_True_rs_41": "score"})
score_df_['n_folds'] = 10
score_df_['shuffle'] = True
score_df_['rs'] = 41
score_df = score_df.append(score_df_)

save_results= "exp8_1.csv"
score_df_ = load_cv_results(save_results)
score_df_['filename'] = save_results
score_df_ = score_df_.rename(index=str, columns={"cv_score_n_folds_10_shuffle_True_rs_41": "score"})
score_df_['n_folds'] = 10
score_df_['shuffle'] = True
score_df_['rs'] = 41
score_df = score_df.append(score_df_)

save_results= "exp8_2.csv"
score_df_ = load_cv_results(save_results)
score_df_['filename'] = save_results
score_df_ = score_df_.rename(index=str, columns={"cv_score_n_folds_6_shuffle_False_rs_41": "score"})
score_df_['n_folds'] = 6
score_df_['shuffle'] = False
score_df_['rs'] = 41
score_df = score_df.append(score_df_)

save_results= "exp9.csv"
score_df_ = load_cv_results(save_results)
score_df_['filename'] = save_results
score_df_ = score_df_.rename(index=str, columns={"cv_score_n_folds_5_shuffle_False_rs_None": "score"})
score_df_['n_folds'] = 5
score_df_['shuffle'] = False
score_df_['rs'] = None
score_df = score_df.append(score_df_)

score_df.reset_index(drop=True, inplace=True)
display(score_df)
score_df = score_df.sort_values(by="score", axis=0)

In [ ]:
display(score_df)

### Set Model Parameters

In [ ]:
model_indices = [1731, 1634]
random_seeds = [0, 1]
n_folds = None
shuffle = False

### Load Models from CSV File, Train, and Create Submission Files

In [ ]:
for model_index in model_indices:
    for rs in random_seeds:
        # retrieve top scoring row
        best = score_df.loc[model_index]
        display(best)

        # retrieve model parameters from pandas row
        alg = best["alg"]
        params_json = best["params_json"]
        fs = best["feature_set"]

        print("alg: {}".format(alg))
        print("params_json: {}".format(params_json))
        print("feature_set: {}".format(fs))

        # retrieve relevant values
        alg_cls = alg_map[alg]
        params = json.loads(params_json)
        params.pop("bagging_seed", None)
        params["random_state"] = rs

        # train algorithm
        model, preds = train_model(params=params, fs=fs, n_fold=n_folds, shuffle=shuffle, alg=alg, test_eval=True)
        
        files_dir = os.path.dirname(inspect.getfile(kaggle_files))
        submission_dir = os.path.join(files_dir, "submission")
        sample_submission_file = os.path.join(submission_dir, "sample_submission.csv")
        submission = pd.read_csv(sample_submission_file, index_col='seg_id')
        submission['time_to_failure'] = preds
        print(submission.head())
        submission.to_csv('submission_4194_n_folds_'+str(n_folds)+"_shuffle_"+str(shuffle)+"_"+str(model_index)+'_seed_'+str(rs)+'.csv')